In [1]:
%%time
import numpy as np
import random
from copy import deepcopy
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import math,time,sys
from matplotlib import pyplot
import pandas as pd
from datetime import datetime
from functools import partial
import seaborn as sns 
from sklearn.metrics import roc_auc_score
from sklearn.neural_network import MLPClassifier
# from sklearn.naive_bayes import 
# from sklearn.ensemble import RanfomForestClassifier
from sklearn.ensemble import RandomForestClassifier

Wall time: 4.59 s


In [2]:
%%time

MaxIter = 20
pop_size = 20
omega = 0.99
bp = 0.5
bsize = 4
wsize = 16

def initialise(partCount, dim, trainX, testX, trainy, testy):    
    population=np.zeros((partCount,dim))
    minn = 1
    maxx = math.floor(0.5*dim)
    
    if maxx<minn:
        maxx = minn + 1
        #not(c[i].all())
    
    for i in range(partCount):
        random.seed(i**3 + 10 + time.time() ) 
        no = random.randint(minn,maxx)
        if no == 0:
            no = 1
        random.seed(time.time()+ 100)
        pos = random.sample(range(0,dim-1),no)
        for j in pos:
            population[i][j]=1
            
    return population

def fitness(agent, trainX, testX, trainy, testy):
    # print(agent)
    cols=np.flatnonzero(agent)
    # print(cols)
    val=1
    if np.shape(cols)[0]==0:
        return val  
    clf = RandomForestClassifier(n_estimators=50)
    #clf=KNeighborsClassifier(n_neighbors=50)
    #clf=MLPClassifier(alpha=0.001, hidden_layer_sizes=(1000,500,100),max_iter=2000,random_state=4)
    train_data=trainX[:,cols]
    test_data=testX[:,cols]
    clf.fit(train_data,trainy)
    val=1-clf.score(test_data,testy)

    #in case of multi objective  []
    set_cnt=sum(agent)
    set_cnt=set_cnt/np.shape(agent)[0]
    val=omega*val+(1-omega)*set_cnt
    return val

def allfit(population, trainX, testX, trainy, testy):
    x=np.shape(population)[0]
    acc=np.zeros(x)
    for i in range(x):
        acc[i]=fitness(population[i],trainX, testX, trainy, testy)     
        #print(acc[i])
    return acc

def test_accuracy(agent, trainX, testX, trainy, testy):
    cols=np.flatnonzero(agent)
    val=1
    if np.shape(cols)[0]==0:
        return val    
    clf = RandomForestClassifier(n_estimators=50)
    #clf=MLPClassifier(alpha=0.001, hidden_layer_sizes=(1000,500,100),max_iter=2000,random_state=4)
    #clf=KNeighborsClassifier(n_neighbors=50)
    # clf=MLPClassifier( alpha=0.01, max_iterno=1000) #hidden_layer_sizes=(1000,500,100)
    #cross=4
    #test_size=(1/cross)
    #X_train, X_test, y_train, y_test = train_test_split(trainX, trainy,  stratify=trainy,test_size=test_size)
    train_data=trainX[:,cols]
    test_data=testX[:,cols]
    clf.fit(train_data,trainy)
    val=clf.score(test_data,testy)
    return val

def onecnt(agent):
    return sum(agent)

def sigmoid(gamma):
    if gamma < 0:
        return 1 - 1/(1 + math.exp(gamma))
    else:
        return 1/(1 + math.exp(-gamma))
    
    
    
    
    


Wall time: 0 ns


In [3]:
def HS(pop, fit, dimension, trainX, testX, trainy, testy):    
    
    hybrid = np.array([])
    counter = 0

    for j in range(dimension):
        random.seed(j**3 + 10 + time.time())
        ra = random.randint(0, pop_size-1)
        hybrid = np.append(hybrid, pop[ra][j])

    worst = pop[0]
    for j in range(pop_size):
        if(fit[j] > fitness(worst, trainX, testX, trainy, testy)):
            worst = deepcopy(pop[j])
            counter = j

    if(fitness(worst, trainX, testX, trainy, testy) > fitness(hybrid, trainX, testX, trainy, testy)):
        fit[counter] = deepcopy(fitness(hybrid, trainX, testX, trainy, testy))
        pop[counter] = deepcopy(hybrid)

    return pop, fit

    
def NMRA(dataset):
    df = pd.read_csv(dataset)
    a, b = np.shape(df)
    data = df.values[:,0:b-1]
    label = df.values[:,b-1]
    dimension = data.shape[1]
    
    cross = 5
    test_size = (1/cross)
    trainX, testX, trainy, testy = train_test_split(data, label,stratify=label ,test_size=test_size,random_state=(7+17*int(time.time()%1000)))

    clf = RandomForestClassifier(n_estimators=50)
    clf.fit(trainX,trainy)

    val=clf.score(testX,testy)
    whole_accuracy = val
    print("Total Acc: ",val)
    
    pop = initialise(pop_size, dimension, trainX, testX, trainy, testy)
    fit = allfit(pop, trainX, testX, trainy, testy)
    
    ind = np.argsort(fit)
    index_b, index_w = [], []

    for i in range(bsize):
        index_b.append(ind[i])
        
    for i in range(wsize):
        index_w.append(ind[i])
    
    bestpop = pop[ind[0]]
    bestfit = fit[ind[0]]
    
    for i in range(MaxIter):
        for j in range(pop_size):
            pop, fit = HS(pop, fit, dimension, trainX, testX, trainy, testy)
        
        
        
        
        for j in range(len(index_w)):
            l = []
            for k in range(dimension):
                random.seed(time.time()+k*2)
                l.append(random.random())
                
            pos = random.sample(range(0,len(index_w)-1),2)
            s = np.add(pop[index_w[j]], np.multiply(l, np.subtract(pop[index_w[pos[0]]],pop[index_w[pos[1]]])))
            
            if(fitness(s,trainX, testX, trainy, testy) < fit[index_w[j]]):
                fit[index_w[j]] = fitness(s,trainX, testX, trainy, testy)
                pop[index_w[j]] = s.copy()
                
                
        for j in range(len(index_b)):
            random.seed(time.time())
            if(random.random() < bp): 
                l, nl = [], []
                for k in range(dimension):
                    random.seed(time.time()+k*2)
                    l.append(random.random())

                for k in l:
                    nl.append(1-k)

                s = np.add(np.multiply(nl, pop[index_b[j]]), np.multiply(l, (np.subtract(bestpop, pop[index_b[j]]))))

                if(fitness(s,trainX, testX, trainy, testy) < fit[index_b[j]]):
                    fit[index_b[j]] = fitness(s,trainX, testX, trainy, testy)
                    pop[index_b[j]] = s.copy()  
                    
        for j in range(pop_size):
            for k in range(dimension):
                random.seed(time.time())
                if (sigmoid(pop[j][k]) > 0.5):
                    pop[j][k] = 1
                    
                else:
                    pop[j][k] = 0
                
                
        fit = allfit(pop, trainX, testX, trainy, testy)
        ind = np.argsort(fit)
        index_b, index_w = [], []

        for i in range(bsize):
            index_b.append(ind[i])

        for i in range(wsize):
            index_w.append(ind[i])

        bestpop = pop[ind[0]]
        bestfit = fit[ind[0]]
    
    fit = allfit(pop, trainX, testX, trainy, testy)
    ind = np.argsort(fit)
    bestpop = pop[ind[0]]
    bestfit = fit[ind[0]]
    
    gbest = bestpop[0]
    for i in range(len(pop)):
        if (fitness(pop[i], trainX, testX, trainy, testy) < fitness(gbest, trainX, testX, trainy, testy)):
            gbest = deepcopy(pop[i])
    
    
    testAcc = test_accuracy(bestpop, trainX, testX, trainy, testy)
    featCnt = onecnt(bestpop)
    #print("best agent: ", bestpop)
    print("Test Accuracy: ", testAcc)
    print("#Features: ", featCnt)
            
    return testAcc, featCnt, bestpop

In [4]:
%%time
Ta,fc,bp = NMRA('IITM128.csv')

Total Acc:  0.9692307692307692
Test Accuracy:  0.963076923076923
#Features:  87.0
Wall time: 3h 41min 9s


In [5]:
bp

array([1., 1., 1., 1., 1., 0., 0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1.,
       0., 0., 1., 0., 1., 0., 1., 1., 0., 0., 1., 1., 0., 1., 0., 1., 1.,
       1., 1., 0., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1.,
       0., 1., 1., 1., 1., 0., 1., 0., 1., 0., 1., 0., 0., 1., 0., 1., 1.,
       1., 1., 0., 1., 0., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 0., 1.,
       1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 0.,
       1., 1., 0., 1., 1., 1., 1., 1., 0.])

In [7]:
List = []

for i in range(0,len(bp)):
    if bp[i] == 1:
        List.append(i)
        
df_train = pd.read_csv("IITM128.csv")
y = (df_train['label'])
df_train1 = df_train[df_train.columns[List]]
l = []
for i in range(0,len(List)):
    l.append(i)
    
df_train1.columns = l

df_train1['label'] = y
df_train1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


,0,1,2,3,4,5,6,7,8,9,...,78,79,80,81,82,83,84,85,86,label
0,0.009862,0.011445,0.005392,0.001609,0.000516,0.048293,0.844509,1.892785,0.583005,0.167826,...,0.000035,0.000058,0.000229,0.000238,0.000488,0.000551,0.000330,0.000254,0.000186,Assamese
1,0.000180,0.000934,0.000941,0.007122,0.038213,0.577700,0.434400,0.332166,0.875804,0.374820,...,0.000139,0.000288,0.000561,0.000966,0.000420,0.000471,0.000441,0.000196,0.000147,Assamese
2,0.000322,0.001019,0.001124,0.018744,0.107425,0.391567,0.058652,0.285366,0.906578,0.844500,...,0.000059,0.000116,0.000214,0.000318,0.000150,0.000183,0.000190,0.000131,0.000048,Assamese
3,0.000416,0.000962,0.002874,0.035911,0.188132,0.699432,0.101537,0.304569,0.600820,1.438876,...,0.000014,0.000081,0.000329,0.000366,0.000219,0.000151,0.000180,0.000242,0.000305,Assamese
4,0.016845,0.046121,0.050567,0.278968,1.727229,4.493108,6.378520,1.567698,1.897226,1.165529,...,0.000486,0.000693,0.000706,0.000615,0.000264,0.000245,0.000159,0.000139,0.000419,Assamese
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3245,0.039602,0.016221,0.010791,0.045113,0.374282,8.345696,16.802679,16.385887,20.768005,6.230567,...,0.002938,0.003456,0.002945,0.002165,0.001306,0.000754,0.000368,0.000104,0.000010,Telugu
3246,0.149945,0.070613,0.016239,0.052733,0.622141,11.021016,27.996975,15.667769,4.889742,2.230314,...,0.005599,0.007398,0.006048,0.005397,0.003318,0.002020,0.000938,0.000227,0.000011,Telugu
3247,0.549227,0.358309,0.153024,0.153383,1.099469,9.914358,26.537991,5.615302,2.793209,2.330279,...,0.007095,0.008343,0.008005,0.008298,0.006930,0.005167,0.001888,0.000514,0.000028,Telugu
3248,0.303473,0.193823,0.065367,0.135263,0.653783,7.623892,15.936741,15.815204,9.590445,4.106088,...,0.007039,0.008723,0.009638,0.011048,0.007675,0.005179,0.002411,0.000744,0.000040,Telugu


In [8]:
df_train1.to_csv('IITM_Spec128_NMRAHS.csv')